In [1]:
#remove cell visibility
from IPython.display import HTML
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.input').hide()
    } else {
        $('div.input').show()
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Toggle cell visibility <a href="javascript:code_toggle()">here</a>.''')
display(tag)

In [2]:
%matplotlib inline
import control
import numpy
import sympy as sym
from IPython.display import display, Markdown
import ipywidgets as widgets
import matplotlib.pyplot as plt


#print a matrix latex-like
def bmatrix(a):
     """Returns a LaTeX bmatrix - by Damir Arbula (ICCT project)

     :a: numpy array
     :returns: LaTeX bmatrix as a string
     """
     if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
     lines = str(a).replace('[', '').replace(']', '').splitlines()
     rv = [r'\begin{bmatrix}']
     rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
     rv +=  [r'\end{bmatrix}']
     return '\n'.join(rv)


# Display formatted matrix: 
def vmatrix(a):
    if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{vmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{vmatrix}']
    return '\n'.join(rv)


#matrixWidget is a matrix looking widget built with a VBox of HBox(es) that returns a numPy array as value !
class matrixWidget(widgets.VBox):
    def updateM(self,change):
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.M_[irow,icol] = self.children[irow].children[icol].value
                #print(self.M_[irow,icol])
        self.value = self.M_

    def dummychangecallback(self,change):
        pass
    
    
    def __init__(self,n,m):
        self.n = n
        self.m = m
        self.M_ = numpy.matrix(numpy.zeros((self.n,self.m)))
        self.value = self.M_
        widgets.VBox.__init__(self,
                             children = [
                                 widgets.HBox(children = 
                                              [widgets.FloatText(value=0.0, layout=widgets.Layout(width='90px')) for i in range(m)]
                                             ) 
                                 for j in range(n)
                             ])
        
        #fill in widgets and tell interact to call updateM each time a children changes value
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
                self.children[irow].children[icol].observe(self.updateM, names='value')
        #value = Unicode('example@example.com', help="The email value.").tag(sync=True)
        self.observe(self.updateM, names='value', type= 'All')
        
    def setM(self, newM):
        #disable callbacks, change values, and reenable
        self.unobserve(self.updateM, names='value', type= 'All')
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].unobserve(self.updateM, names='value')
        self.M_ = newM
        self.value = self.M_
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].observe(self.updateM, names='value')
        self.observe(self.updateM, names='value', type= 'All')        

                #self.children[irow].children[icol].observe(self.updateM, names='value')

             
#overlaod class for state space systems that DO NOT remove "useless" states (what "professor" of automatic control would do this?)
class sss(control.StateSpace):
    def __init__(self,*args):
        #call base class init constructor
        control.StateSpace.__init__(self,*args)
    #disable function below in base class
    def _remove_useless_states(self):
        pass

## Controllo della profondità di un veicolo subacqueo autonomo (AUV)

La dinamica dell'angolo di beccheggio $\theta$ di un veicolo subacqueo autonomo (AUV) può essere modellata come:
$$
J_y\ddot{\theta}=−D\dot{\theta}+CV\gamma,
$$
dove $J_y=8.333$ kg$\text{m}^2$ è il momento di inerzia del veicolo attorno all'asse di beccheggio, $D = 30$ Nms il coefficiente di resistenza aerodinamica, $C=45$ Ns il coefficiente di efficacia della pinna caudale, $\gamma$ l'angolo della pinna caudale (espresso in radianti e da mantenere entro $\pm10°$) e $V$ velocità di crociera rispetto alla corrente in m/s.

La velocità verticale $w$ di un AUV può essere modellata approssimativamente come:
$$
w = V\theta.
$$

Per motivi di sicurezza, l'angolo di inclinazione non deve mai superare $\pm20°$ durante il funzionamento. L'angolo di inclinazione e la profondità ($z$) vengono misurati da un filtro di navigazione.

Le equazioni dinamiche nella forma di stato sono
\begin{cases}
    \dot{x} = \begin{bmatrix} -D/J_y & 0 & 0 \\ 1 & 0 & 0 \\ 0 & V & 0 \end{bmatrix}x + \begin{bmatrix} CV/J_y \\ 0 \\ 0 \end{bmatrix}u\\
    y = \begin{bmatrix} 0 & 1 & 0 \\ 0 & 0 & 1 \end{bmatrix}x,
\end{cases}

dove $x=\begin{bmatrix} x_1 & x_2 & x_3 \end{bmatrix}^T=\begin{bmatrix} \dot{\theta} & \theta & z \end{bmatrix}^T$ e $u=\gamma$.

Considerando la costante $V=1$ m/s, si vuole sviluppare un regolatore per $z$ che permette di avere errore di regime nullo in risposta ad una richiesta di posizione a gradino.

### Design del regolatore
#### Design del controller

Per soddisfare i requisiti, si aggiunge prima un nuovo stato:
$$
\dot{x_4} = z-y_d = x_3 - y_d
$$
Il sistema aumentato risultante è quindi:
\begin{cases}
    \dot{x_a} = \begin{bmatrix} -D/J_y & 0 & 0 & 0 \\ 1 & 0 & 0 & 0 \\ 0 & V & 0 & 0 \\ 0 & 0 & 1 & 0 \end{bmatrix}x_a + \begin{bmatrix} CV/J_y & 0 \\ 0 & 0 \\ 0 & 0 \\ 0 & -1 \end{bmatrix} \begin{bmatrix} u \\ y_d \end{bmatrix}\\
    y = \begin{bmatrix} 0 & 1 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 1 \end{bmatrix}x_a,
\end{cases}

dove $x_a = \begin{bmatrix} x_1 & x_2 & x_3 & x_4 \end{bmatrix}^T = \begin{bmatrix} \dot{\theta} & \theta & z & x_4 \end{bmatrix}^T$.

Con il primo ingresso $u$, poiché il sistema è controllabile, si posizionano i poli in modo da stabilizzare il sistema e rispettare i limiti su $u$ ($\pm10°$) e $\theta$ ($\pm20°$) quando si dà un gradino di $y_d=1$ m. Una possibile soluzione è posizionare i poli in $-5.9$, $-0.9$ e $-0.63\pm0.70i$.

#### Design dell'osservatore

Si hanno a disposizione le misure di $x_2$, $x_3$ e $x_4$, quindi si deve stimare solo $x_1$. Si può lavorare con il sottosistema
$x_l =\begin{bmatrix} x_1 & x_2 \end{bmatrix}^T$, $\dot{x_l}=A_lx_l + B_lu$ e $y_l = x_2 = C_lx_l$. Definendo $L=\begin{bmatrix} l_1 & l_2 \end{bmatrix}^T$ si possono calcolare gli autovalori $\lambda_1$ e $\lambda_2$ di $A_l-LC_l$ e risolvere per $l_1$ e $l_2$:
\begin{cases}
l_1 = \lambda_1\lambda_2 + D\lambda_1/J_y + D\lambda_2/J_y + D^2/J_y^2 \\
l_2 = -\lambda_1 - \lambda_2 - D/J_y \, .
\end{cases}

Si scelgono $\lambda_1 = \lambda_2 = -15$.

### Come usare questo notebook?
- Prova, cambiando i poli del controller, a soddisfare i requisiti con $V = 0.75$ m/s.
- Verifica il comportamento del regolatore in presenza di un errore iniziale nella stima dello stato e cerca di migliorarne le prestazioni.

In [3]:
# Preparatory cell

X0 = numpy.matrix('0.0')
K = numpy.matrix([0,0,0,0])
L = numpy.matrix([[0],[0]])

X0w = matrixWidget(1,1)
X0w.setM(X0)
Kw = matrixWidget(1,4)
Kw.setM(K)
Lw = matrixWidget(2,1)
Lw.setM(L)


eig1c = matrixWidget(1,1)
eig2c = matrixWidget(2,1)
eig3c = matrixWidget(1,1)
eig4c = matrixWidget(2,1)
eig1c.setM(numpy.matrix([-5.7])) 
eig2c.setM(numpy.matrix([[-0.63],[-0.70]]))
eig3c.setM(numpy.matrix([-0.9]))
eig4c.setM(numpy.matrix([[-0.63],[-0.70]]))

eig1o = matrixWidget(1,1)
eig2o = matrixWidget(2,1)
eig1o.setM(numpy.matrix([-15.])) 
eig2o.setM(numpy.matrix([[-15.],[0.]]))

In [4]:
# Misc

#create dummy widget 
DW = widgets.FloatText(layout=widgets.Layout(width='0px', height='0px'))

#create button widget
START = widgets.Button(
    description='Test',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Test',
    icon='check'
)
                       
def on_start_button_clicked(b):
    #This is a workaround to have intreactive_output call the callback:
    #   force the value of the dummy widget to change
    if DW.value> 0 :
        DW.value = -1
    else: 
        DW.value = 1
    pass
START.on_click(on_start_button_clicked)

# Define type of method 
selm = widgets.Dropdown(
    options= [('Imposta K e L','Set K and L'), ('Imposta gli autovalori','Set the eigenvalues')],
    value= 'Set the eigenvalues',
    description='',
    disabled=False
)

# Define the number of complex eigenvalues
selec = widgets.Dropdown(
    options= [('0 autovalori complessi','0 complex eigenvalues'), ('2 autovalori complessi','2 complex eigenvalues'), ('4 autovalori complessi','4 complex eigenvalues')],
    value= '2 complex eigenvalues',
    description='Autov. controller:',
    style = {'description_width': 'initial'},
    disabled=False
)
seleo = widgets.Dropdown(
    options= [('0 autovalori complessi','0 complex eigenvalues'), ('2 autovalori complessi','2 complex eigenvalues')],
    value= '0 complex eigenvalues',
    description='Autov. osservatore:',
    style = {'description_width': 'initial'},
    disabled=False
)

#define type of ipout 
selu = widgets.Dropdown(
    options=[('impulso','impulse'), ('gradino','step'), ('sinusoide','sinusoid'), ('onda quadra','square wave')],
    value='step',
    description='Riferimento:',
    style = {'description_width': 'initial'},
    disabled=False
)
# Define the values of the input
u = widgets.FloatSlider(
    value=1,
    min=0,
    max=4,
    step=0.1,
    description='',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
v = widgets.FloatSlider(
    value=1,
    min=0.5,
    max=4,
    step=0.1,
    description=r'$V$:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
period = widgets.FloatSlider(
    value=0.5,
    min=0.001,
    max=10,
    step=0.001,
    description='Periodo: ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

simTime = widgets.FloatText(
    value=10,
    description='',
    disabled=False
)

In [5]:
# Support functions

def eigen_choice(selec,seleo):
    if selec == '0 complex eigenvalues':
        eig1c.children[0].children[0].disabled = False
        eig2c.children[1].children[0].disabled = True
        eig3c.children[0].children[0].disabled = False
        eig4c.children[0].children[0].disabled = False
        eig4c.children[1].children[0].disabled = True
        eigc = 0
    if seleo == '0 complex eigenvalues':
        eig1o.children[0].children[0].disabled = False
        eig2o.children[1].children[0].disabled = True
        eigo = 0
    if selec == '2 complex eigenvalues':
        eig1c.children[0].children[0].disabled = False
        eig2c.children[1].children[0].disabled = False
        eig3c.children[0].children[0].disabled = False
        eig4c.children[0].children[0].disabled = True
        eig4c.children[1].children[0].disabled = True
        eigc = 2
    if seleo == '2 complex eigenvalues':
        eig1o.children[0].children[0].disabled = True
        eig2o.children[1].children[0].disabled = False
        eigo = 2
    if selec == '4 complex eigenvalues':
        eig1c.children[0].children[0].disabled = True
        eig2c.children[1].children[0].disabled = False
        eig3c.children[0].children[0].disabled = True
        eig4c.children[0].children[0].disabled = False
        eig4c.children[1].children[0].disabled = False
        eigc = 4
    return eigc, eigo

def method_choice(selm):
    if selm == 'Set K and L':
        method = 1
        selec.disabled = True
        seleo.disabled = True
    if selm == 'Set the eigenvalues':
        method = 2
        selec.disabled = False
        seleo.disabled = False
    return method

In [6]:
D = 30
c = 45
V = 1
Jy = 8.333

A = numpy.matrix([[-D/Jy, 0, 0, 0],
                  [1, 0, 0, 0],
                  [0, V, 0, 0],
                  [0, 0, 1, 0]])
Bu = numpy.matrix([[c*V/Jy],[0],[0],[0]])
Bref = numpy.matrix([[0],[0],[0],[-1]])
C = numpy.matrix([0,0,1,0])

Al = numpy.matrix([[-D/Jy, 0],
                  [1, 0]])
Cl = numpy.matrix([0,1])

def main_callback2(v, X0w, K, L, eig1c, eig2c, eig3c, eig4c, eig1o, eig2o, u, period, selm, selec, seleo, selu, simTime, DW):
    eigc, eigo = eigen_choice(selec,seleo)
    method = method_choice(selm)
    
    A = numpy.matrix([[-D/Jy, 0, 0, 0],
                  [1, 0, 0, 0],
                  [0, v, 0, 0],
                  [0, 0, 1, 0]])
    Bu = numpy.matrix([[c*v/Jy],[0],[0],[0]])
    
    if method == 1:
        solc = numpy.linalg.eig(A-Bu*K)
        solo = numpy.linalg.eig(Al-L*Cl)
    if method == 2:
        if eig1c[0,0]==eig2c[0,0] or eig1c[0,0]==eig3c[0,0] or eig1c[0,0]==eig4c[0,0]:
            eig1c[0,0] -= 0.0001
        if eig2c[0,0]==eig3c[0,0] or eig2c[0,0]==eig4c[0,0]:
            eig3c[0,0] -= 0.0002
        if eigc == 0:
            K = control.acker(A, Bu, [eig1c[0,0], eig2c[0,0], eig3c[0,0], eig4c[0,0]])
            Kw.setM(K)
        if eigc == 2:
            K = control.acker(A, Bu, [eig3c[0,0],
                                      eig1c[0,0],
                                      numpy.complex(eig2c[0,0], eig2c[1,0]), 
                                      numpy.complex(eig2c[0,0],-eig2c[1,0])])
            Kw.setM(K)
        if eigc == 4:
            K = control.acker(A, Bu, [numpy.complex(eig4c[0,0], eig4c[1,0]), 
                                      numpy.complex(eig4c[0,0],-eig4c[1,0]),
                                      numpy.complex(eig2c[0,0], eig2c[1,0]), 
                                      numpy.complex(eig2c[0,0],-eig2c[1,0])])
            Kw.setM(K)
        if eigo == 0:
            temp1 = eig1o[0,0]
            temp2 = eig2o[0,0]
            L = numpy.matrix([[temp1*temp2 + D*temp1/Jy + D*temp2/Jy + D**2/Jy**2], [-temp1 - temp2 - D/Jy]]) 
            Lw.setM(L)
        if eigo == 2:
            temp1 = numpy.complex(eig2o[0,0], eig2o[1,0])
            temp2 = numpy.complex(eig2o[0,0],-eig2o[1,0])
            L = numpy.matrix([[temp1*temp2 + D*temp1/Jy + D*temp2/Jy + D**2/Jy**2], [-temp1 - temp2 - D/Jy]]) 
            L = numpy.real(L)
            Lw.setM(L)
            
    sys = sss(A,numpy.hstack((Bu,Bref)),[[0,1,0,0],[0,0,1,0],[0,0,0,1]],[[0,0],[0,0],[0,0]])
    syse = sss(Al-L*Cl,numpy.hstack((Bu[0:2],L)),[1,0],[0,0])
    sysc = sss(0,[0,0,0,0],0,-K)
    sys_append = control.append(sys,syse,sysc)
    sys_CL = control.connect(sys_append,
                             [[1,5],[3,5],[4,1],[5,4],[6,1],[7,2],[8,3]],
                             [2],
                             [2,5])
    
    X0w1 = numpy.zeros((6,1))
    X0w1[4,0] = X0w
    if simTime != 0:
        T = numpy.linspace(0, simTime, 10000)
    else:
        T = numpy.linspace(0, 1, 10000)
      
    if selu == 'impulse': #selu
        U = [0 for t in range(0,len(T))]
        U[0] = u
        T, yout, xout = control.forced_response(sys_CL,T,U,X0w1)
    if selu == 'step':
        U = [u for t in range(0,len(T))]
        T, yout, xout = control.forced_response(sys_CL,T,U,X0w1)
    if selu == 'sinusoid':
        U = u*numpy.sin(2*numpy.pi/period*T)
        T, yout, xout = control.forced_response(sys_CL,T,U,X0w1)
    if selu == 'square wave':
        U = u*numpy.sign(numpy.sin(2*numpy.pi/period*T))
        T, yout, xout = control.forced_response(sys_CL,T,U,X0w1)
    
    try:
        step_info_dict = control.step_info(sys_CL[0,0],SettlingTimeThreshold=0.05,T=T)
        print('Step info: \n\tTempo di salita =',step_info_dict['RiseTime'],'\n\tTempo di assestamento (5%) =',step_info_dict['SettlingTime'],'\n\tOvershoot (%)=',step_info_dict['Overshoot'])
        print('Massimo valore di x2 (% di 20°)=', max(abs(xout[1]))/(numpy.pi/180*20)*100)
        print('Massimo valore di u (% di 10°)=', max(abs(yout[1]))/(numpy.pi/180*10)*100)
    except:
        print("Errore nel calcolo delle step info.")
    
    fig = plt.figure(num='Simulation1', figsize=(14,12))
    
    fig.add_subplot(221)
    plt.title('Risposta dell\'uscita')
    plt.ylabel('Uscita')
    plt.plot(T,yout[0],T,U,'r--')
    plt.xlabel('$t$ [s]')
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.legend(['$y$','Riferimento'])
    plt.grid()
    
    fig.add_subplot(222)
    plt.title('Ingresso')
    plt.ylabel('$u$')
    plt.plot(T,yout[1])
    plt.plot(T,[numpy.pi/180*10 for i in range(len(T))],'r--')
    plt.plot(T,[-numpy.pi/180*10 for i in range(len(T))],'r--')
    plt.xlabel('$t$ [s]')
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.grid()
    
    fig.add_subplot(223)
    plt.title('Risposta degli stati')
    plt.ylabel('Stati')
    plt.plot(T,xout[0],
             T,xout[1],
             T,xout[2],
             T,xout[3],
             T,[numpy.pi/180*20 for i in range(len(T))],'r--',
             T,[-numpy.pi/180*20 for i in range(len(T))],'r--')
    plt.xlabel('$t$ [s]')
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.legend(['$x_{1}$','$x_{2}$','$x_{3}$','$x_{4}$','+lim $x_2$','-lim $x_2$'])
    plt.grid()
    
    fig.add_subplot(224)
    plt.title('Errore di stima')
    plt.ylabel('Errore')
    plt.plot(T,xout[4]-xout[0])
    plt.xlabel('$t$ [s]')
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.legend(['$e_{1}$'])
    plt.grid()
    #plt.tight_layout()
   
alltogether2 = widgets.VBox([widgets.HBox([selm, 
                                          selec,
                                          seleo,
                                          selu]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([widgets.VBox([widgets.Label('K:',border=3), Kw, 
                                                        widgets.Label('Autovalori:',border=3),
                                                        widgets.HBox([eig1c, 
                                                                      eig2c, 
                                                                      eig3c,
                                                                      eig4c])]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([widgets.VBox([widgets.HBox([widgets.Label('L:',border=3), Lw, widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('Autovalori:',border=3), 
                                          eig1o, 
                                          eig2o,
                                          widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('X0 stim.:',border=3), X0w]),
                                          widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                                       widgets.HBox([
                                          widgets.VBox([widgets.Label('Tempo di simulazione [s]:',border=3)]),
                                          widgets.VBox([simTime])])])]),
                                          widgets.Label(' ',border=3)]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([widgets.Label('Riferimento [m]:',border=3),u,
                                          v]),
                            widgets.HBox([period,START])])
out2 = widgets.interactive_output(main_callback2, {'v':v, 'X0w':X0w, 'K':Kw, 'L':Lw,
                                                   'eig1c':eig1c, 'eig2c':eig2c, 'eig3c':eig3c, 'eig4c':eig4c, 
                                                   'eig1o':eig1o, 'eig2o':eig2o, 
                                                   'u':u, 'period':period, 'selm':selm, 'selec':selec, 'seleo':seleo, 'selu':selu, 'simTime':simTime, 'DW':DW})
out2.layout.height = '880px'
display(out2, alltogether2)

Output(layout=Layout(height='880px'))